In [3]:
import os
import sys
import json

def get_latest_json_dump() -> dict:
    data_dir = "data"

    files = os.listdir(data_dir)
    files = [f for f in files if f.endswith(".json")]
    latest = max(files)

    with open(f"{data_dir}/{latest}", "r") as f:
        return json.load(f)


# parse in pandas
import pandas as pd
data = get_latest_json_dump()
df = pd.DataFrame(data)


df.head()

,0,1,2,3,4,5,6,7,8,9,...,430,431,432,433,434,435,436,437,438,439
url,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...,https://www.willhaben.at/iad/immobilien/d/eige...
title,Kompakter 1-Zimmer Wohnhit in Liesing,++++PREISSTURZ - GELEGENHEIT MIT WOHNBAUFÖRDER...,RUHIG GELEGENE STARTER-/SINGLEWOHNUNG IM 1. LI...,#SQ - UNBEFRISTET VERMIETET ALTBAUWOHNUNG IN T...,"Jetzt investieren und profitieren: 1,5-Zimmer-...",Einzigartige Gelegenheit in Top Lage Dornbachs...,1-Zimmer-Altbauwohnung zum selbst gestalten!,# SQ -SANIERUNGSBEDÜRFTIGE HELLE 1 ZIMMER WOH...,Airbnb-Wohnen-Gewerbe | U3 Johnstraße | Weiter...,** WIENER WOHNKULTUR - ALTBAUJUWEL IN FAVORITE...,...,Gemütliche Single-Wohnung in Wien 1140!,Effizientes Cityapartment mit guter Aufteilung...,35M2 UNSANIERTE 2-ZIMMERWOHNUNG IN DER ALBRECH...,Großzügige Loggiawohnung in Jedlesee!,"Klein, aber fein: Renovierungsbedürftiges Stad...",2 Zimmer EIGENTUMSWOHNUNG in Top-Lage 1220 Wi...,WOHNGLÜCK AUF 30 m² | KLEIN - FEIN - UND ALLES...,Klein aber fein - Meine Garçonnière in Toplage,TOP-PREIS,*** PROVISIONSFREI *** NIEDRIG-ENERGIE *** GRO...
last_change,"Zuletzt geändert: 02.04.2024, 17:35 Uhr","Zuletzt geändert: 01.04.2024, 17:55 Uhr","Zuletzt geändert: 02.04.2024, 05:04 Uhr","Zuletzt geändert: 28.03.2024, 12:00 Uhr","Zuletzt geändert: 03.04.2024, 13:57 Uhr","Zuletzt geändert: 03.04.2024, 11:18 Uhr","Zuletzt geändert: 05.04.2024, 08:30 Uhr","Zuletzt geändert: 01.04.2024, 09:55 Uhr","Zuletzt geändert: 02.04.2024, 05:07 Uhr","Zuletzt geändert: 02.04.2024, 04:10 Uhr",...,"Zuletzt geändert: 28.03.2024, 09:40 Uhr","Zuletzt geändert: 02.04.2024, 04:15 Uhr","Zuletzt geändert: 04.04.2024, 16:15 Uhr","Zuletzt geändert: 25.03.2024, 07:15 Uhr","Zuletzt geändert: 29.03.2024, 14:15 Uhr","Zuletzt geändert: 02.04.2024, 04:20 Uhr","Zuletzt geändert: 03.04.2024, 14:37 Uhr","Zuletzt geändert: 03.04.2024, 14:40 Uhr","Zuletzt geändert: 02.04.2024, 04:32 Uhr","Zuletzt geändert: 26.03.2024, 20:35 Uhr"
price_info,"{'Kaufpreis': '€ 129.000', 'Sonstige Kosten (e...","{'Kaufpreis': '€ 145.000', 'Sonstige Kosten (e...","{'Kaufpreis': '€ 127.000', 'Sonstige Kosten (e...","{'Kaufpreis': '€ 135.000', 'Monatliche Kosten ...","{'Kaufpreis': '€ 119.005', 'Maklerprovision: '...","{'Kaufpreis': '€ 99.000', 'Maklerprovision: ':...","{'Kaufpreis': '€ 99.000', 'Sonstige Kosten (ex...","{'Kaufpreis': '€ 110.000', 'Monatliche Kosten ...","{'Kaufpreis': '€ 80.600', 'Monatliche Kosten (...","{'Kaufpreis': '€ 110.000', 'Monatliche Kosten ...",...,"{'Kaufpreis': '€ 129.000', 'Sonstige Kosten (e...","{'Kaufpreis': '€ 128.000', 'Heizkosten (exkl. ...","{'Kaufpreis': '€ 119.000', 'Maklerprovision: '...","{'Kaufpreis': '€ 150.000', 'Monatliche Kosten ...","{'Kaufpreis': '€ 120.650', 'Maklerprovision: '...","{'Kaufpreis': '€ 149.000', 'Sonstige Kosten (e...","{'Kaufpreis': '€ 135.000', 'Betriebskosten (in...","{'Kaufpreis': '€ 135.000', 'Betriebskosten (in...","{'Kaufpreis': '€ 136.000', 'Sonstige Kosten (e...","{'Kaufpreis': '€ 102.719,56'}"
address,"1230 Wien, 23. Bezirk, Liesing","3465 Königsbrunn am Wagram, Tulln, Niederöster...","Randhartinge